In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICE'] = ''
import random

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
from omegaconf import OmegaConf

%load_ext autoreload
%autoreload 2

from importlib.util import find_spec
if find_spec("text_recognizer") is None:
    import sys
    sys.path.append('..')

In [2]:
from hydra import compose, initialize
from omegaconf import OmegaConf
from hydra.utils import instantiate

In [3]:
path = "../training/conf/network/conv_transformer.yaml"

In [9]:
with open(path, "rb") as f:
    cfg = OmegaConf.load(f)

In [10]:
cfg

{'_target_': 'text_recognizer.networks.ConvTransformer', 'input_dims': [1, 1, 576, 640], 'hidden_dim': 128, 'num_classes': 58, 'pad_index': 3, 'encoder': {'_target_': 'text_recognizer.networks.EfficientNet', 'arch': 'b0', 'stochastic_dropout_rate': 0.2, 'bn_momentum': 0.99, 'bn_eps': 0.001, 'depth': 5, 'out_channels': 128, 'stride': [2, 1]}, 'decoder': {'_target_': 'text_recognizer.networks.transformer.Decoder', 'depth': 6, 'block': {'_target_': 'text_recognizer.networks.transformer.DecoderBlock', 'self_attn': {'_target_': 'text_recognizer.networks.transformer.Attention', 'dim': 128, 'num_heads': 8, 'dim_head': 64, 'dropout_rate': 0.4, 'causal': True, 'rotary_embedding': {'_target_': 'text_recognizer.networks.transformer.RotaryEmbedding', 'dim': 64}}, 'cross_attn': {'_target_': 'text_recognizer.networks.transformer.Attention', 'dim': 128, 'num_heads': 8, 'dim_head': 64, 'dropout_rate': 0.4, 'causal': False}, 'norm': {'_target_': 'text_recognizer.networks.transformer.RMSNorm', 'dim': 12

In [11]:
net = instantiate(cfg)

In [12]:
from torchinfo import summary

In [13]:
summary(net, ((1, 1, 576, 640), (1, 682)), device="cpu", depth=4)

Layer (type:depth-idx)                                       Output Shape              Param #
ConvTransformer                                              [1, 58, 682]              --
├─EfficientNet: 1-1                                          [1, 128, 18, 80]          850,044
│    └─Sequential: 2-1                                       [1, 16, 288, 320]         --
│    │    └─ZeroPad2d: 3-1                                   [1, 1, 577, 641]          --
│    │    └─Conv2d: 3-2                                      [1, 16, 288, 320]         144
│    │    └─BatchNorm2d: 3-3                                 [1, 16, 288, 320]         32
│    │    └─Mish: 3-4                                        [1, 16, 288, 320]         --
│    └─ModuleList: 2                                         --                        --
│    │    └─MBConvBlock: 3-5                                 [1, 16, 288, 320]         --
│    │    │    └─Depthwise: 4-1                              [1, 16, 288, 320]         17